<h1>Cousera Capstone Project<h1>
    Comparing Cities in Midwestern United States

# Importing/Installing Necessary Packages

In [42]:
import pandas as pd
import numpy as np
!pip install beautifulsoup4
from bs4 import BeautifulSoup

from pandas.io.json import json_normalize
from sklearn.cluster import KMeans

import requests
!pip install wget
import wget
import urllib.request
from urllib.request import urlopen

!pip install geopy
from geopy.geocoders import Nominatim
import folium
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.style.use('ggplot')
import matplotlib.cm as cm
import matplotlib.colors as colors

<p> Reading in Wikipedia Page and Converting to Pandas Dataframe <p>

In [47]:
path="https://raw.githubusercontent.com/Chrisbutz20/Coursera_Capstone/master/Project_Data_1.csv"
data=pd.read_csv(path,error_bad_lines=False)
data.head()

,City,State,% Growth (2000 - 2013),Population,Latitude,Longitude
0,Chicago,Illinois,-6.1,"2,718,782.00",41.878114,-87.629798
1,Indianapolis,Indiana,7.8,"843,393.00",39.768403,-86.158068
2,Columbus,Ohio,14.8,"822,553.00",39.961176,-82.998794
3,Detroit,Michigan,-27.1,"688,701.00",42.331427,-83.045754
4,Milwaukee,Wisconsin,0.3,"599,164.00",43.038902,-87.906474


In [48]:
data.columns

Index(['City', 'State', '% Growth (2000 - 2013)', 'Population', 'Latitude',
       'Longitude'],
      dtype='object')

In [49]:
data['Population']=data['Population'].str.replace(',', '')
data['Population']=pd.to_numeric(data['Population'])
data.head()

,City,State,% Growth (2000 - 2013),Population,Latitude,Longitude
0,Chicago,Illinois,-6.1,2718782.0,41.878114,-87.629798
1,Indianapolis,Indiana,7.8,843393.0,39.768403,-86.158068
2,Columbus,Ohio,14.8,822553.0,39.961176,-82.998794
3,Detroit,Michigan,-27.1,688701.0,42.331427,-83.045754
4,Milwaukee,Wisconsin,0.3,599164.0,43.038902,-87.906474


In [50]:
data.set_index('City')

,State,% Growth (2000 - 2013),Population,Latitude,Longitude
City,,,,,
Chicago,Illinois,-6.1,2718782.0,41.878114,-87.629798
Indianapolis,Indiana,7.8,843393.0,39.768403,-86.158068
Columbus,Ohio,14.8,822553.0,39.961176,-82.998794
Detroit,Michigan,-27.1,688701.0,42.331427,-83.045754
Milwaukee,Wisconsin,0.3,599164.0,43.038902,-87.906474
...,...,...,...,...,...
Lincoln Park,Michigan,-6.7,37313.0,42.250594,-83.178536
Calumet City,Illinois,-4.5,37240.0,41.615591,-87.529487
Muskegon,Michigan,-7.1,37213.0,43.234181,-86.248392


In [51]:
data.describe()

,% Growth (2000 - 2013),Population,Latitude,Longitude
count,229.000000,2.290000e+02,229.000000,229.000000
mean,9.025764,1.055604e+05,41.739755,-88.548359
std,22.651288,2.063043e+05,2.032251,4.452187
min,-27.100000,3.688800e+04,37.084227,-101.292291
25%,-1.700000,4.713400e+04,40.110588,-92.342578
50%,3.400000,6.007000e+04,41.845588,-88.019826
75%,13.100000,8.389100e+04,42.606409,-85.139351
max,203.600000,2.718782e+06,48.232967,-80.649519


In [52]:
data_stats=data.groupby(['State'])
data_stats['Population'].describe()

,count,mean,std,min,25%,50%,75%,max
State,,,,,,,,
Illinois,52.0,116452.673077,369607.534231,37240.0,41720.25,54156.0,75676.00,2718782.0
Indiana,21.0,113974.857143,173202.192727,45775.0,53665.00,70316.0,83891.00,843393.0
Iowa,13.0,79822.307692,45288.246745,40566.0,58253.00,61969.0,82459.00,207510.0
Kansas,11.0,120655.909091,99999.327171,41889.0,53243.50,90811.0,140184.00,386552.0
Michigan,31.0,96105.387097,115128.554492,37213.0,51075.50,72952.0,95546.00,688701.0
Minnesota,24.0,85656.208333,83855.262721,39167.0,49994.75,62353.0,75083.50,400070.0
Missouri,16.0,115247.062500,116571.918368,38816.0,51969.50,72358.0,115767.00,467007.0
Nebraska,4.0,201826.000000,185636.494453,50550.0,52884.75,161200.5,310141.75,434353.0
North Dakota,4.0,70486.250000,30008.954379,46321.0,52779.25,60983.0,78690.00,113658.0


In [53]:
data.columns=data.columns.str.strip()

In [59]:
clean_df=data.copy()
clean_df.set_index('City')

,State,% Growth (2000 - 2013),Population,Latitude,Longitude
City,,,,,
Chicago,Illinois,-6.1,2718782.0,41.878114,-87.629798
Indianapolis,Indiana,7.8,843393.0,39.768403,-86.158068
Columbus,Ohio,14.8,822553.0,39.961176,-82.998794
Detroit,Michigan,-27.1,688701.0,42.331427,-83.045754
Milwaukee,Wisconsin,0.3,599164.0,43.038902,-87.906474
...,...,...,...,...,...
Lincoln Park,Michigan,-6.7,37313.0,42.250594,-83.178536
Calumet City,Illinois,-4.5,37240.0,41.615591,-87.529487
Muskegon,Michigan,-7.1,37213.0,43.234181,-86.248392


Create Map of Midwest with Labels for each City <br/>
Format: City (Growth)

In [214]:
map_ctr='Dubuque,Iowa'
geolocator=Nominatim(user_agent="Foursquare_agent")
location=geolocator.geocode(map_ctr)
t_lat=location.latitude
t_long=location.longitude

In [215]:
map_mdw=folium.Map(location=[location.latitude,location.longitude],zoom_start=5)

for lat,long,neighbour,growth in zip(clean_df.Latitude,clean_df.Longitude,clean_df.City,clean_df['% Growth (2000 - 2013)']):
    label='{} ({})'.format(neighbour,growth)
    label=folium.Popup(label,parse_html=True)
    folium.features.CircleMarker(
        [lat,long],
        radius=3,
        color='blue',
        popup=label,
        fill=False
    ).add_to(map_mdw)
    
map_mdw

## Four Square API: Neighbourhood Venue Analysis ##

Credentials

In [216]:
CLIENT_ID = 'J3BXVIECWLAHZXARVRCBB2YM5UHI2AMJURPE1GOOMAJEF4RK' # your Foursquare ID
CLIENT_SECRET = 'JNVENONWCBDNZZ3UG5RTBGJ5TXMCXOPWC0GL3VIQ0AAAG0MC' # your Foursquare Secret
VERSION = '20200806' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: J3BXVIECWLAHZXARVRCBB2YM5UHI2AMJURPE1GOOMAJEF4RK
CLIENT_SECRET:JNVENONWCBDNZZ3UG5RTBGJ5TXMCXOPWC0GL3VIQ0AAAG0MC


-

New Function "getNearbyVenues" returns venues within the radius of each neighbourhood's Coorindates

In [217]:
def getNearbyVenues(names, latitudes, longitudes, radius=3000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]["groups"][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['City', 
                  'City Latitude', 
                  'City Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [219]:
LIMIT=2500
df_venues=getNearbyVenues(names=clean_df.City,
                                   latitudes=clean_df.Latitude,
                                   longitudes=clean_df.Longitude)

Chicago
Indianapolis
Columbus
Detroit
Milwaukee
Kansas City
Omaha
Minneapolis
Cleveland
Wichita
St. Louis
Cincinnati
St. Paul
Toledo
Lincoln
Fort Wayne
Madison
Des Moines
Aurora
Akron
Grand Rapids
Overland Park
Springfield
Rockford
Kansas City
Joliet
Naperville
Dayton
Warren
Olathe
Sterling Heights
Cedar Rapids
Topeka
Evansville
Independence
Ann Arbor
Springfield
Peoria
Columbia
Lansing
Fargo
Rochester
Elgin
Green Bay
Davenport
South Bend
Kenosha
Flint
Dearborn
Livonia
Lee's Summit
Lawrence
Waukegan
Bloomington
Duluth
Carmel
Cicero
Fishers
Champaign
Troy
O'Fallon
Westland
Bloomington
Sioux City
Farmington Hills
Parma
Hammond
Bloomington
Gary
Brooklyn Park
Racine
St. Joseph
Arlington Heights
Evanston
Kalamazoo
Schaumburg
Decatur
Wyoming
Plymouth
Bolingbrook
Appleton
Southfield
Rochester Hills
Canton
Iowa City
Waukesha
Lafayette
Muncie
Palatine
Waterloo
St. Charles
Eau Claire
Bismarck
Oshkosh
St. Cloud
Woodbury
Eagan
Maple Grove
Youngstown
Skokie
Shawnee
Janesville
Lorain
Eden Prairie
Ha

In [220]:
df_venues.shape

(20115, 7)

In [221]:
df_venues.groupby('City').count()

,City Latitude,City Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
City,,,,,,
Addison,47,47,47,47,47,47
Akron,100,100,100,100,100,100
Ames,100,100,100,100,100,100
Anderson,81,81,81,81,81,81
Ankeny,100,100,100,100,100,100
...,...,...,...,...,...,...
Wheeling,95,95,95,95,95,95
Wichita,100,100,100,100,100,100
Woodbury,100,100,100,100,100,100


Using One-Hot Encoding to format Venue Category for easier analysis

In [222]:
# one hot encoding
df_onehot = pd.get_dummies(df_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
df_onehot['City'] = df_venues['City'] 

# move neighborhood column to the first column
fixed_columns = [df_onehot.columns[-1]] + list(df_onehot.columns[:-1])
df_onehot = df_onehot[fixed_columns]

Calculating frequency of each category for each neighbourhood

In [223]:
df_grouped=df_onehot.groupby('City').mean().reset_index()

In [226]:
top_count=10

for city in df_grouped['City']:
    print("----"+city+"----")
    temp = df_grouped[df_grouped['City'] == city].T.reset_index()
    temp.columns = ['Venue','Freq']
    temp = temp.iloc[1:]
    temp['Freq'] = temp['Freq'].astype(float)
    temp = temp.round({'Freq': 2})
    print(temp.sort_values('Freq', ascending=False).reset_index(drop=True).head(top_count))
    print('\n')

----Addison----
                  Venue  Freq
0    Mexican Restaurant  0.15
1           Pizza Place  0.13
2           Golf Course  0.06
3         Hot Dog Joint  0.06
4        Ice Cream Shop  0.04
5         Grocery Store  0.04
6  Gym / Fitness Center  0.02
7         Big Box Store  0.02
8        Breakfast Spot  0.02
9         Bowling Alley  0.02


----Akron----
                 Venue  Freq
0       Sandwich Place  0.05
1                  Bar  0.05
2  American Restaurant  0.04
3                 Café  0.04
4          Coffee Shop  0.04
5          Pizza Place  0.03
6   Italian Restaurant  0.03
7              Brewery  0.03
8   Mexican Restaurant  0.03
9                Trail  0.03


----Ames----
                  Venue  Freq
0           Pizza Place  0.07
1                   Bar  0.05
2           Coffee Shop  0.05
3  Fast Food Restaurant  0.04
4  Gym / Fitness Center  0.03
5        Sandwich Place  0.03
6              Dive Bar  0.03
7   American Restaurant  0.03
8         Grocery Store  0.03
9   

New function to return most common venue type for each city.

In [227]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Calculating most common venue type for each neighborhood in Toronto

In [228]:
num_top_venues = 20

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['City']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
venues_sorted = pd.DataFrame(columns=columns)
venues_sorted['City'] = df_grouped['City']

for ind in np.arange(df_grouped.shape[0]):
    venues_sorted.iloc[ind, 1:] = return_most_common_venues(df_grouped.iloc[ind, :], num_top_venues)

venues_sorted.head()

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,...,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
0,Addison,Mexican Restaurant,Pizza Place,Hot Dog Joint,Golf Course,Ice Cream Shop,Grocery Store,Convention Center,Business Service,Bar,...,Liquor Store,Gas Station,Bowling Alley,Breakfast Spot,Coffee Shop,Motorcycle Shop,Bakery,Sandwich Place,Dog Run,Rental Car Location
1,Akron,Bar,Sandwich Place,Café,American Restaurant,Coffee Shop,Brewery,Italian Restaurant,Trail,Mexican Restaurant,...,Pizza Place,Sports Bar,Music Venue,Indie Movie Theater,Zoo,Food Service,Gay Bar,Art Gallery,Fast Food Restaurant,Middle Eastern Restaurant
2,Ames,Pizza Place,Coffee Shop,Bar,Fast Food Restaurant,Dive Bar,Sandwich Place,Gym / Fitness Center,Grocery Store,American Restaurant,...,Mexican Restaurant,Thai Restaurant,Food Truck,Bank,Liquor Store,Bakery,Italian Restaurant,Lingerie Store,Golf Course,Rock Club
3,Anderson,Fast Food Restaurant,Pizza Place,Discount Store,American Restaurant,Pharmacy,Mexican Restaurant,Ice Cream Shop,Jewelry Store,Golf Course,...,Café,Video Store,Burger Joint,Bar,Sandwich Place,Food,Donut Shop,Racetrack,Movie Theater,Steakhouse
4,Ankeny,Mexican Restaurant,Pizza Place,Coffee Shop,Gym,Bar,Sandwich Place,Gym / Fitness Center,American Restaurant,Donut Shop,...,Hotel,Restaurant,Cosmetics Shop,Gas Station,Frozen Yogurt Shop,Grocery Store,Salon / Barbershop,Department Store,Men's Store,Optical Shop


## Begin K-Means Clustering ##

In [229]:
df_cluster=df_grouped.drop(['City'],1)

Identify # of Clusters (10), perform clustering, and attach cluster labels to new df 

In [230]:
k_clusters=10
kmeans=KMeans(n_clusters=k_clusters,n_init=20,max_iter=100,random_state=1).fit(df_cluster)

In [231]:
venues_sorted.insert(0,'Cluster Labels', kmeans.labels_)

In [232]:
df_merge=clean_df
df_merge=df_merge.join(venues_sorted.set_index('City'), on='City')
df_merge['Cluster Labels']=df_merge['Cluster Labels'].astype(int)
df_merge.head()

,City,State,% Growth (2000 - 2013),Population,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,...,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
0,Chicago,Illinois,-6.1,2718782.0,41.878114,-87.629798,8,Hotel,Grocery Store,Park,...,Steakhouse,Sushi Restaurant,American Restaurant,Mediterranean Restaurant,Music Venue,Cocktail Bar,Garden,Sandwich Place,Mexican Restaurant,Gastropub
1,Indianapolis,Indiana,7.8,843393.0,39.768403,-86.158068,8,Bar,Hotel,Mexican Restaurant,...,New American Restaurant,Grocery Store,Plaza,Café,Steakhouse,Concert Hall,Breakfast Spot,Pizza Place,Park,Deli / Bodega
2,Columbus,Ohio,14.8,822553.0,39.961176,-82.998794,1,American Restaurant,Park,Bar,...,Fast Food Restaurant,Italian Restaurant,Convenience Store,Deli / Bodega,Rental Car Location,Café,Theater,Cocktail Bar,Bakery,Pharmacy
3,Detroit,Michigan,-27.1,688701.0,42.331427,-83.045754,8,Coffee Shop,Park,Hotel,...,Restaurant,Lounge,Diner,Bar,Burger Joint,Italian Restaurant,Outdoor Sculpture,Café,Waterfront,New American Restaurant
4,Milwaukee,Wisconsin,0.3,599164.0,43.038902,-87.906474,8,Hotel,Coffee Shop,Park,...,Burger Joint,Bar,BBQ Joint,Museum,New American Restaurant,Sandwich Place,Sushi Restaurant,Other Great Outdoors,Concert Hall,Wine Bar


In [233]:
df_cluster=df_merge.drop({'Latitude','Longitude'},axis=1)
df_cluster.groupby('Cluster Labels').describe()

% Growth (2000 - 2013)                                      \
                                count       mean        std   min     25%   
Cluster Labels                                                              
0                                16.0   8.837500  14.403512 -11.4  -0.025   
1                                24.0   8.537500  18.392834 -10.1  -0.275   
2                                36.0   8.205556  16.980946  -8.7  -0.625   
3                                50.0  17.590000  36.471063  -5.9   0.600   
4                                24.0  12.791667  17.358569  -6.7   1.900   
5                                 1.0 -23.400000        NaN -23.4 -23.400   
6                                12.0   2.158333   8.500530  -4.4  -3.150   
7                                35.0   9.802857  13.826477 -13.5   0.050   
8                                14.0   0.657143  18.628502 -27.1  -7.850   
9                                17.0  -6.829412   8.490934 -20.2 -10.300   

                                     Population                                \
                  50%     75%    max      count           mean            std   
Cluster Labels                                                                  
0                4.25  12.850   40.4       16.0   76785.687500   45392.662700   
1                3.50  11.475   85.7       24.0  102664.500000  160836.143674   
2                2.90  13.225   79.5       36.0   55983.888889   23126.790020   
3                5.35  16.250  203.6       50.0   63388.840000   26139.706251   
4                6.10  22.650   60.4       24.0   59346.000000   14812.688838   
5              -23.40 -23.400  -23.4        1.0   78450.000000            NaN   
6                0.15   4.250   26.6       12.0   77470.500000   29171.541317   
7                5.90  19.800   41.5       35.0  113457.028571  109821.779125   
8               -1.25   4.350   53.3       14.0  524356.571429  669542.724972   
9               -6.30  -2.300    7.8       17.0   91268.882353   61386.715375   

                                                                    
                    min        25%       50%        75%        max  
Cluster Labels                                                      
0               37240.0   43324.75   58391.0   94155.75   199963.0  
1               36888.0   42387.75   57671.0   79392.25   822553.0  
2               37313.0   40781.00   47811.5   62621.00   147806.0  
3               37530.0   47232.00   56580.0   75722.25   181260.0  
4               37159.0   48887.50   56401.0   67480.00    93184.0  
5               78450.0   78450.00   78450.0   78450.00    78450.0  
6               38877.0   55871.00   72488.5  102812.50   128429.0  
7               37839.0   51604.50   71016.0  115894.50   467007.0  
8               45929.0  216468.50  307966.5  549390.50  2718782.0  
9               40768.0   51265.00   65184.0  117006.00   282313.0

In [234]:
df_cluster.sort_values('Population')

,City,State,% Growth (2000 - 2013),Population,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,...,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
228,Beloit,Wisconsin,2.9,36888.0,1,Bar,American Restaurant,Pizza Place,Video Store,Park,...,Grocery Store,Discount Store,Gas Station,Italian Restaurant,Rental Car Location,Antique Shop,Furniture / Home Store,Restaurant,Liquor Store,Bookstore
227,Greenfield,Wisconsin,4.8,37159.0,4,Cosmetics Shop,Clothing Store,Pizza Place,Department Store,Coffee Shop,...,Supermarket,Sushi Restaurant,Fast Food Restaurant,Dive Bar,Italian Restaurant,Restaurant,Thrift / Vintage Store,Toy / Game Store,Ice Cream Shop,Bakery
226,Muskegon,Michigan,-7.1,37213.0,1,Park,Bar,Pizza Place,American Restaurant,Coffee Shop,...,Brewery,Mexican Restaurant,Bank,Sandwich Place,Boat or Ferry,Pharmacy,Liquor Store,Theater,Disc Golf,Baseball Field
225,Calumet City,Illinois,-4.5,37240.0,0,Fast Food Restaurant,Mexican Restaurant,Discount Store,Pharmacy,Liquor Store,...,Fried Chicken Joint,Bank,Burger Joint,Post Office,Bar,Diner,Clothing Store,Video Game Store,Women's Store,Bowling Alley
224,Lincoln Park,Michigan,-6.7,37313.0,2,Pizza Place,American Restaurant,Ice Cream Shop,Fast Food Restaurant,Discount Store,...,Chinese Restaurant,Bar,Burger Joint,Park,Pharmacy,Sandwich Place,Gas Station,Coffee Shop,Liquor Store,Grocery Store
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,Milwaukee,Wisconsin,0.3,599164.0,8,Hotel,Coffee Shop,Park,Spa,Steakhouse,...,Burger Joint,Bar,BBQ Joint,Museum,New American Restaurant,Sandwich Place,Sushi Restaurant,Other Great Outdoors,Concert Hall,Wine Bar
3,Detroit,Michigan,-27.1,688701.0,8,Coffee Shop,Park,Hotel,Plaza,American Restaurant,...,Restaurant,Lounge,Diner,Bar,Burger Joint,Italian Restaurant,Outdoor Sculpture,Café,Waterfront,New American Restaurant
2,Columbus,Ohio,14.8,822553.0,1,American Restaurant,Park,Bar,Coffee Shop,Hotel,...,Fast Food Restaurant,Italian Restaurant,Convenience Store,Deli / Bodega,Rental Car Location,Café,Theater,Cocktail Bar,Bakery,Pharmacy
1,Indianapolis,Indiana,7.8,843393.0,8,Bar,Hotel,Mexican Restaurant,Burger Joint,American Restaurant,...,New American Restaurant,Grocery Store,Plaza,Café,Steakhouse,Concert Hall,Breakfast Spot,Pizza Place,Park,Deli / Bodega


# Create Map with Cluster Labels 

In [235]:
# create map
map_clusters = folium.Map(location=[location.latitude, location.longitude], zoom_start=5)

# set color scheme for the clusters
x = np.arange(k_clusters)
ys = [i + x + (i*x)**2 for i in range(k_clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df_merge['Latitude'], df_merge['Longitude'], df_merge['City'], df_merge['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# Separate Clusters for Easier Analysis

In [236]:
%%markdown
## Cluster 0

## Cluster 0


In [237]:
df0=df_merge.loc[df_merge['Cluster Labels'] == 0]
df0

,City,State,% Growth (2000 - 2013),Population,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,...,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
18,Aurora,Illinois,38.4,199963.0,41.760585,-88.320071,0,Mexican Restaurant,Discount Store,Pizza Place,...,Grocery Store,Storage Facility,Rental Car Location,Brewery,Bridal Shop,Donut Shop,Breakfast Spot,Motel,Brazilian Restaurant,Supplement Shop
29,Olathe,Kansas,40.4,131885.0,38.881396,-94.819129,0,Pizza Place,Park,Fried Chicken Joint,...,Juice Bar,Auto Garage,Baseball Field,Convenience Store,American Restaurant,BBQ Joint,Disc Golf,Big Box Store,Motorcycle Shop,Burger Joint
32,Topeka,Kansas,3.4,127679.0,39.055823,-95.689019,0,Mexican Restaurant,Fast Food Restaurant,Pizza Place,...,Rental Car Location,Chinese Restaurant,Café,Burger Joint,Brewery,Breakfast Spot,ATM,Discount Store,Hotel,Ice Cream Shop
42,Elgin,Illinois,16.0,110145.0,42.035408,-88.282567,0,Mexican Restaurant,Pizza Place,Park,...,Convenience Store,Discount Store,Soccer Field,Café,Baseball Field,Sports Bar,Liquor Store,Supermarket,Restaurant,Golf Course
52,Waukegan,Illinois,0.5,88826.0,42.363633,-87.844794,0,Mexican Restaurant,Fast Food Restaurant,Bar,...,Grocery Store,Fried Chicken Joint,Hotel,Breakfast Spot,Chinese Restaurant,Gym,Theater,Music Store,Pub,Clothing Store
56,Cicero,Illinois,-1.6,84103.0,41.845588,-87.753945,0,Mexican Restaurant,Pizza Place,Pharmacy,...,Bank,Sandwich Place,Ice Cream Shop,Gym / Fitness Center,Dessert Shop,Park,American Restaurant,Café,Stadium,Supermarket
111,Terre Haute,Indiana,2.5,61025.0,39.466703,-87.413909,0,Mexican Restaurant,Sandwich Place,Bar,...,Fast Food Restaurant,Fountain,Restaurant,Cajun / Creole Restaurant,Baseball Stadium,Café,Other Repair Shop,Donut Shop,Lawyer,Gas Station
115,Pontiac,Michigan,-11.4,59887.0,42.638922,-83.291047,0,Fast Food Restaurant,Discount Store,Pizza Place,...,Automotive Shop,Cosmetics Shop,Music Venue,Bookstore,Big Box Store,Optical Shop,Gay Bar,Electronics Store,Golf Course,Burger Joint
125,Kokomo,Indiana,21.3,56895.0,40.486427,-86.133603,0,Discount Store,Pizza Place,Mexican Restaurant,...,Ice Cream Shop,Bank,Coffee Shop,Gym / Fitness Center,Pet Store,Lingerie Store,Park,Sushi Restaurant,Gas Station,Jewelry Store
126,Berwyn,Illinois,5.1,56758.0,41.850587,-87.793669,0,Pizza Place,Fast Food Restaurant,Ice Cream Shop,...,Pool,Café,Fried Chicken Joint,BBQ Joint,Brewery,Italian Restaurant,Donut Shop,Pub,Gym / Fitness Center,Sushi Restaurant


In [238]:
# create map
cluster0 = folium.Map(location=[location.latitude, location.longitude], zoom_start=5)

# set color scheme for the clusters
x = np.arange(k_clusters)
ys = [i + x + (i*x)**2 for i in range(k_clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df0['Latitude'], df0['Longitude'], df0['City'], df0['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(cluster0)
       
cluster0

In [239]:
%%markdown

## Cluster 1


## Cluster 1


In [240]:
df1=df_merge.loc[df_merge['Cluster Labels'] == 1]
df1

,City,State,% Growth (2000 - 2013),Population,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,...,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
2,Columbus,Ohio,14.8,822553.0,39.961176,-82.998794,1,American Restaurant,Park,Bar,...,Fast Food Restaurant,Italian Restaurant,Convenience Store,Deli / Bodega,Rental Car Location,Café,Theater,Cocktail Bar,Bakery,Pharmacy
15,Fort Wayne,Indiana,1.0,256496.0,41.079273,-85.139351,1,Bar,Mexican Restaurant,Pizza Place,...,Discount Store,Taco Place,Gas Station,Bakery,Greek Restaurant,Beer Garden,Theater,Hotel,Fried Chicken Joint,Juice Bar
23,Rockford,Illinois,-1.0,150251.0,42.271131,-89.093995,1,Discount Store,Bar,Fast Food Restaurant,...,Café,Restaurant,New American Restaurant,Asian Restaurant,Video Store,History Museum,Brewery,Fried Chicken Joint,Italian Restaurant,Historic Site
33,Evansville,Indiana,-0.8,120310.0,37.971559,-87.571090,1,Pizza Place,Bar,Sandwich Place,...,Ice Cream Shop,Donut Shop,Performing Arts Venue,Pub,Hotel,Park,Gay Bar,Mexican Restaurant,Liquor Store,Strip Club
46,Kenosha,Wisconsin,9.5,99889.0,42.584742,-87.821185,1,American Restaurant,Bar,Park,...,Brewery,Bakery,Italian Restaurant,Fast Food Restaurant,Sandwich Place,Beach,Event Space,Pub,Storage Facility,General Entertainment
54,Duluth,Minnesota,-0.1,86128.0,46.786672,-92.100485,1,Pizza Place,Brewery,Hotel,...,Mobile Phone Shop,Ice Cream Shop,Italian Restaurant,Clothing Store,Garden,Coffee Shop,Boutique,Rental Car Location,Breakfast Spot,Seafood Restaurant
71,St. Joseph,Missouri,4.1,77147.0,39.767458,-94.846681,1,Pizza Place,Park,Convenience Store,...,Pub,ATM,Casino,Theater,Sports Bar,Historic Site,Lake,Stadium,Lawyer,Steakhouse
80,Appleton,Wisconsin,4.5,73596.0,44.261931,-88.415385,1,Bar,American Restaurant,Pizza Place,...,Coffee Shop,Pub,Indian Restaurant,Gym / Fitness Center,Gastropub,Burger Joint,Liquor Store,Greek Restaurant,Lounge,Japanese Restaurant
90,St. Charles,Missouri,11.3,67569.0,38.788106,-90.497436,1,American Restaurant,Bar,Breakfast Spot,...,Café,Italian Restaurant,Trail,Brewery,Bakery,Fried Chicken Joint,Historic Site,Grocery Store,Dessert Shop,Pub
91,Eau Claire,Wisconsin,8.7,67545.0,44.811349,-91.498494,1,Pizza Place,Bar,American Restaurant,...,Brewery,Bagel Shop,Pub,Convenience Store,Grocery Store,Mexican Restaurant,Gym,Pharmacy,BBQ Joint,ATM


In [241]:
# create map
cluster1 = folium.Map(location=[location.latitude, location.longitude], zoom_start=5)

# set color scheme for the clusters
x = np.arange(k_clusters)
ys = [i + x + (i*x)**2 for i in range(k_clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df1['Latitude'], df1['Longitude'], df1['City'], df1['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(cluster1)
       
cluster1

In [242]:
%%markdown

# Cluster 2


# Cluster 2


In [243]:
df2=df_merge.loc[df_merge['Cluster Labels'] == 2]
df2

,City,State,% Growth (2000 - 2013),Population,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,...,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
25,Joliet,Illinois,36.5,147806.0,41.525031,-88.081725,2,Fast Food Restaurant,Discount Store,Pharmacy,...,Coffee Shop,Bar,American Restaurant,Gas Station,Dive Bar,Hot Dog Joint,Pub,Currency Exchange,Cocktail Bar,Supermarket
34,Independence,Missouri,3.2,117240.0,39.091116,-94.415507,2,Fast Food Restaurant,Pizza Place,Pharmacy,...,Bar,BBQ Joint,Gas Station,Bowling Alley,Mexican Restaurant,Burger Joint,Ice Cream Shop,Cosmetics Shop,Toy / Game Store,Salon / Barbershop
66,Hammond,Indiana,-4.6,78967.0,41.583369,-87.500041,2,Ice Cream Shop,Fast Food Restaurant,Pizza Place,...,Mobile Phone Shop,Pharmacy,Liquor Store,Salon / Barbershop,Gym / Fitness Center,Sporting Goods Shop,Burger Joint,Supermarket,Grocery Store,Big Box Store
69,Brooklyn Park,Minnesota,16.0,78373.0,45.094132,-93.356341,2,Sandwich Place,Pizza Place,Gas Station,...,Department Store,Gym / Fitness Center,Burger Joint,Shipping Store,Cajun / Creole Restaurant,Rental Service,Café,Rental Car Location,College Rec Center,Convenience Store
76,Decatur,Illinois,-8.7,74710.0,39.840315,-88.954800,2,Pizza Place,Sandwich Place,American Restaurant,...,Restaurant,New American Restaurant,Clothing Store,Bank,Supermarket,Park,Pub,Sporting Goods Shop,BBQ Joint,Thai Restaurant
77,Wyoming,Michigan,6.5,74100.0,42.913360,-85.705309,2,Pizza Place,Mexican Restaurant,Park,...,Bank,Intersection,Fast Food Restaurant,Hotel,Vietnamese Restaurant,Video Store,Convenience Store,Music Store,Dry Cleaner,Laundromat
79,Bolingbrook,Illinois,29.7,73936.0,41.698642,-88.068395,2,Fast Food Restaurant,Pizza Place,Hotel,...,Asian Restaurant,Mexican Restaurant,Sports Bar,Intersection,Coffee Shop,American Restaurant,Rental Car Location,Convenience Store,Storage Facility,Bus Stop
89,Waterloo,Iowa,-0.5,68366.0,42.492786,-92.342578,2,Pizza Place,Discount Store,Convenience Store,...,Fast Food Restaurant,Bank,Lounge,Sports Bar,Italian Restaurant,Coffee Shop,Dive Bar,Video Store,ATM,Rental Car Location
102,Lorain,Ohio,-7.2,63710.0,41.452819,-82.182375,2,Discount Store,Pizza Place,Fast Food Restaurant,...,Liquor Store,Beach,Bank,Grocery Store,Construction & Landscaping,Seafood Restaurant,Greek Restaurant,Mexican Restaurant,Lighthouse,Eye Doctor
104,Hamilton,Ohio,2.7,62258.0,39.399501,-84.561335,2,Fast Food Restaurant,Pizza Place,Convenience Store,...,Discount Store,Donut Shop,American Restaurant,Ice Cream Shop,Chinese Restaurant,Breakfast Spot,Supermarket,Bowling Alley,Brewery,Basketball Stadium


In [244]:
# create map
cluster2 = folium.Map(location=[location.latitude, location.longitude], zoom_start=5)

# set color scheme for the clusters
x = np.arange(k_clusters)
ys = [i + x + (i*x)**2 for i in range(k_clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df2['Latitude'], df2['Longitude'], df2['City'], df2['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(cluster2)
       
cluster2

In [245]:
%%markdown

## Cluster 3


## Cluster 3


In [246]:
df3=df_merge.loc[df_merge['Cluster Labels'] == 3]
df3

,City,State,% Growth (2000 - 2013),Population,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,...,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
21,Overland Park,Kansas,19.4,181260.0,38.982228,-94.670792,3,Sandwich Place,Bakery,Chinese Restaurant,...,Convenience Store,Salon / Barbershop,Italian Restaurant,Bookstore,Thai Restaurant,Toy / Game Store,Coffee Shop,Discount Store,Donut Shop,Supplement Shop
26,Naperville,Illinois,12.0,144864.0,41.750839,-88.153535,3,Pizza Place,Mexican Restaurant,Coffee Shop,...,Baseball Field,BBQ Joint,Bar,Frozen Yogurt Shop,Bookstore,Kitchen Supply Store,Grocery Store,Park,Video Store,Yoga Studio
49,Livonia,Michigan,-5.4,95208.0,42.368370,-83.352710,3,Pizza Place,Park,Diner,...,Mobile Phone Shop,Video Game Store,Italian Restaurant,Comic Shop,Bowling Alley,Sporting Goods Shop,Convenience Store,Fast Food Restaurant,Wine Shop,Middle Eastern Restaurant
51,Lawrence,Kansas,12.7,90811.0,38.971669,-95.235250,3,Pizza Place,Fast Food Restaurant,Bar,...,Breakfast Spot,Fried Chicken Joint,American Restaurant,Brewery,Park,Hotel,Ice Cream Shop,Gas Station,Intersection,Clothing Store
53,Bloomington,Minnesota,1.3,86319.0,44.840798,-93.298280,3,Coffee Shop,Mexican Restaurant,Pizza Place,...,Bakery,Bar,Donut Shop,Theater,Park,Pharmacy,Chinese Restaurant,Bookstore,Restaurant,Café
57,Fishers,Indiana,114.8,83891.0,39.956755,-86.013350,3,Pizza Place,Mexican Restaurant,Coffee Shop,...,Liquor Store,Brewery,Dessert Shop,Deli / Bodega,Grocery Store,Ice Cream Shop,Thai Restaurant,Park,Shoe Store,Chinese Restaurant
59,Troy,Michigan,2.2,82821.0,42.606409,-83.149775,3,Sandwich Place,Pharmacy,Pizza Place,...,Sushi Restaurant,Salon / Barbershop,Gym / Fitness Center,Pet Store,Ice Cream Shop,Gas Station,Thai Restaurant,Chinese Restaurant,Bank,Bakery
60,O'Fallon,Missouri,62.6,82809.0,38.810608,-90.699848,3,Pizza Place,Pharmacy,American Restaurant,...,Baseball Stadium,Discount Store,Convenience Store,Deli / Bodega,Other Great Outdoors,Thai Restaurant,Furniture / Home Store,Breakfast Spot,Steakhouse,Mexican Restaurant
61,Westland,Michigan,-4.7,82578.0,42.324204,-83.400211,3,Pizza Place,Sandwich Place,Coffee Shop,...,Furniture / Home Store,Greek Restaurant,Grocery Store,Asian Restaurant,Multiplex,Gas Station,Chinese Restaurant,General Entertainment,Video Store,Cosmetics Shop
62,Bloomington,Indiana,16.1,82575.0,39.165325,-86.526386,3,Coffee Shop,Mexican Restaurant,Pizza Place,...,Bakery,Bar,Donut Shop,Theater,Park,Pharmacy,Chinese Restaurant,Bookstore,Restaurant,Café


In [247]:
# create map
cluster3 = folium.Map(location=[location.latitude, location.longitude], zoom_start=5)

# set color scheme for the clusters
x = np.arange(k_clusters)
ys = [i + x + (i*x)**2 for i in range(k_clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df3['Latitude'], df3['Longitude'], df3['City'], df3['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(cluster3)
       
cluster3

In [248]:
%%markdown 

## Cluster 4


## Cluster 4


In [249]:
df4=df_merge.loc[df_merge['Cluster Labels'] == 4]
df4

,City,State,% Growth (2000 - 2013),Population,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,...,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
50,Lee's Summit,Missouri,31.2,93184.0,38.910841,-94.382172,4,Cosmetics Shop,Coffee Shop,Bar,...,Department Store,Furniture / Home Store,BBQ Joint,Breakfast Spot,Mexican Restaurant,Steakhouse,Pizza Place,Convenience Store,Thai Restaurant,Brewery
55,Carmel,Indiana,60.4,85927.0,39.978371,-86.118043,4,American Restaurant,Pizza Place,Ice Cream Shop,...,Spa,Mexican Restaurant,Cajun / Creole Restaurant,Coffee Shop,Sporting Goods Shop,Steakhouse,Pub,Yoga Studio,Deli / Bodega,Art Gallery
64,Farmington Hills,Michigan,-0.9,81295.0,42.498994,-83.367717,4,Café,Hotel,Mexican Restaurant,...,Museum,Theater,Bank,Clothing Store,Japanese Restaurant,Golf Course,Diner,Men's Store,Electronics Store,Supermarket
78,Plymouth,Minnesota,12.0,73987.0,45.010519,-93.455509,4,Coffee Shop,Pizza Place,Sandwich Place,...,Ice Cream Shop,Grocery Store,Sushi Restaurant,Deli / Bodega,Gym,Hotel,Gym / Fitness Center,Burger Joint,Spa,Café
81,Southfield,Michigan,-6.7,73006.0,42.473369,-83.221873,4,Sandwich Place,Cosmetics Shop,Discount Store,...,Supermarket,Fast Food Restaurant,Ice Cream Shop,Mobile Phone Shop,Grocery Store,Burger Joint,Thai Restaurant,Liquor Store,Park,Pharmacy
82,Rochester Hills,Michigan,5.7,72952.0,42.658366,-83.149932,4,Coffee Shop,Sandwich Place,Pub,...,Fast Food Restaurant,Cosmetics Shop,Department Store,Pet Store,Seafood Restaurant,Paper / Office Supplies Store,Mediterranean Restaurant,Pizza Place,Tex-Mex Restaurant,Juice Bar
95,Woodbury,Minnesota,39.8,65656.0,44.923855,-92.959380,4,Coffee Shop,Sandwich Place,Pizza Place,...,Mexican Restaurant,Shoe Store,Bakery,Spa,Liquor Store,Jewelry Store,Juice Bar,Grocery Store,Ice Cream Shop,Hotel
97,Maple Grove,Minnesota,27.3,65415.0,45.072464,-93.455788,4,Clothing Store,Hotel,Furniture / Home Store,...,Golf Course,Playground,Pizza Place,Video Store,Gym,Grocery Store,Sandwich Place,Cosmetics Shop,Gym / Fitness Center,Brewery
99,Skokie,Illinois,2.8,65176.0,42.032403,-87.741625,4,Bakery,Mediterranean Restaurant,Bagel Shop,...,Breakfast Spot,Lingerie Store,Liquor Store,Sushi Restaurant,Clothing Store,Donut Shop,Pizza Place,Park,Mexican Restaurant,American Restaurant
109,Burnsville,Minnesota,1.9,61434.0,44.767742,-93.277723,4,Sandwich Place,Coffee Shop,American Restaurant,...,Liquor Store,Burger Joint,Breakfast Spot,Video Store,Vietnamese Restaurant,Storage Facility,Italian Restaurant,Cosmetics Shop,Gym / Fitness Center,Mexican Restaurant


In [250]:
# create map
cluster4 = folium.Map(location=[location.latitude, location.longitude], zoom_start=5)

# set color scheme for the clusters
x = np.arange(k_clusters)
ys = [i + x + (i*x)**2 for i in range(k_clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df4['Latitude'], df4['Longitude'], df4['City'], df4['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(cluster4)
       
cluster4

In [251]:
%%markdown

## Cluster 5


## Cluster 5


In [252]:
df5=df_merge.loc[df_merge['Cluster Labels'] == 5]
df5

,City,State,% Growth (2000 - 2013),Population,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,...,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
68,Gary,Indiana,-23.4,78450.0,41.59337,-87.346427,5,Fried Chicken Joint,Discount Store,Fast Food Restaurant,...,Video Store,Pharmacy,Sports Bar,Department Store,Field,Event Space,Exhibit,Eye Doctor,Fabric Shop,Factory


In [253]:
# create map
cluster5 = folium.Map(location=[location.latitude, location.longitude], zoom_start=5)

# set color scheme for the clusters
x = np.arange(k_clusters)
ys = [i + x + (i*x)**2 for i in range(k_clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df5['Latitude'], df5['Longitude'], df5['City'], df5['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(cluster5)
       
cluster5

In [254]:
%%markdown

## Cluster 6


## Cluster 6


In [255]:
df6=df_merge.loc[df_merge['Cluster Labels'] == 6]
df6

,City,State,% Growth (2000 - 2013),Population,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,...,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
31,Cedar Rapids,Iowa,5.4,128429.0,41.977880,-91.665623,6,Bar,Park,Coffee Shop,...,Grocery Store,Café,Sports Bar,Drugstore,Restaurant,Performing Arts Venue,Breakfast Spot,Farmers Market,Ice Cream Shop,Fast Food Restaurant
39,Lansing,Michigan,-4.4,113972.0,42.732535,-84.555535,6,Bar,Coffee Shop,Chinese Restaurant,...,Breakfast Spot,Golf Course,Bakery,Brewery,Restaurant,Grocery Store,History Museum,Thai Restaurant,Candy Store,Yoga Studio
43,Green Bay,Wisconsin,1.9,104779.0,44.519159,-88.019826,6,Bar,Mexican Restaurant,Harbor / Marina,...,Smoke Shop,Farmers Market,Sandwich Place,Coffee Shop,Park,Hotel,Convenience Store,Pharmacy,Nightclub,Pier
44,Davenport,Iowa,3.9,102157.0,41.523644,-90.577637,6,Bar,American Restaurant,Brewery,...,Park,Theater,Discount Store,BBQ Joint,Grocery Store,Gym,Dive Bar,Cocktail Bar,Sports Bar,Rental Car Location
63,Sioux City,Iowa,-2.9,82459.0,42.499994,-96.400307,6,Bar,Mexican Restaurant,Coffee Shop,...,Thai Restaurant,Pharmacy,Fast Food Restaurant,Discount Store,American Restaurant,Hotel,Ice Cream Shop,Gym / Fitness Center,Concert Hall,Steakhouse
70,Racine,Wisconsin,-4.4,78199.0,42.726131,-87.782852,6,Bar,Mexican Restaurant,Pizza Place,...,Deli / Bodega,Grocery Store,Gastropub,Beach,Thai Restaurant,Music Store,Tattoo Parlor,Korean Restaurant,Exhibit,Restaurant
93,Oshkosh,Wisconsin,5.3,66778.0,44.024706,-88.542614,6,Bar,American Restaurant,Pizza Place,...,Discount Store,Thrift / Vintage Store,Coffee Shop,Sports Bar,Flower Shop,Building,Butcher,Comic Shop,Brewery,Filipino Restaurant
112,West Allis,Wisconsin,-0.6,60697.0,43.016681,-88.007031,6,Bar,Pizza Place,Italian Restaurant,...,Grocery Store,Theme Park,Dive Bar,Racetrack,Latin American Restaurant,German Restaurant,Southern / Soul Food Restaurant,Burger Joint,Peruvian Restaurant,Fair
121,Dubuque,Iowa,0.9,58253.0,42.500558,-90.664572,6,Bar,Pizza Place,American Restaurant,...,Restaurant,Pub,ATM,Park,Discount Store,Nightclub,Mexican Restaurant,Ice Cream Shop,Hotel,Boat or Ferry
159,Sheboygan,Wisconsin,-3.9,48725.0,43.750828,-87.714530,6,Bar,Pizza Place,Coffee Shop,...,Brewery,Burger Joint,Sandwich Place,Chinese Restaurant,Diner,Optical Shop,Gastropub,Discount Store,Beach,Lake


In [256]:
# create map
cluster6 = folium.Map(location=[location.latitude, location.longitude], zoom_start=5)

# set color scheme for the clusters
x = np.arange(k_clusters)
ys = [i + x + (i*x)**2 for i in range(k_clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df6['Latitude'], df6['Longitude'], df6['City'], df6['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(cluster6)
       
cluster6

In [257]:
%%markdown

## Cluster 7


## Cluster 7


In [258]:
df7=df_merge.loc[df_merge['Cluster Labels'] == 7]
df7

,City,State,% Growth (2000 - 2013),Population,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,...,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
5,Kansas City,Missouri,5.5,467007.0,39.099727,-94.578567,7,Mexican Restaurant,Coffee Shop,Brewery,...,Deli / Bodega,Hotel,French Restaurant,History Museum,Sports Bar,Grocery Store,Chinese Restaurant,Seafood Restaurant,New American Restaurant,Gym
6,Omaha,Nebraska,5.9,434353.0,41.252363,-95.997988,7,Pizza Place,Coffee Shop,Bar,...,Mexican Restaurant,Bakery,Furniture / Home Store,Cocktail Bar,Sandwich Place,Farmers Market,Fast Food Restaurant,Pub,Deli / Bodega,Donut Shop
9,Wichita,Kansas,9.7,386552.0,37.688889,-97.336111,7,Bar,Brewery,Coffee Shop,...,Breakfast Spot,Theater,Hotel,Plaza,Thrift / Vintage Store,Mexican Restaurant,Pizza Place,History Museum,Liquor Store,Pharmacy
14,Lincoln,Nebraska,18.0,268738.0,40.825763,-96.685198,7,Mexican Restaurant,Coffee Shop,Bar,...,Steakhouse,Burger Joint,Asian Restaurant,Park,Cocktail Bar,Café,Rock Club,Fast Food Restaurant,Chinese Restaurant,Sports Bar
19,Akron,Ohio,-8.6,198100.0,41.081445,-81.519005,7,Bar,Sandwich Place,Café,...,Pizza Place,Sports Bar,Music Venue,Indie Movie Theater,Zoo,Food Service,Gay Bar,Art Gallery,Fast Food Restaurant,Middle Eastern Restaurant
24,Kansas City,Kansas,1.1,148483.0,39.114053,-94.627464,7,Mexican Restaurant,Coffee Shop,Brewery,...,Deli / Bodega,Hotel,French Restaurant,History Museum,Sports Bar,Grocery Store,Chinese Restaurant,Seafood Restaurant,New American Restaurant,Gym
27,Dayton,Ohio,-13.5,143355.0,39.758948,-84.191607,7,Bar,Pizza Place,Brewery,...,Thrift / Vintage Store,Bakery,Café,Thai Restaurant,Wine Bar,Hot Dog Joint,Pub,New American Restaurant,Neighborhood,Nightclub
35,Ann Arbor,Michigan,2.0,117025.0,42.280826,-83.743038,7,Café,Bar,Park,...,College Auditorium,Grocery Store,Steakhouse,Noodle House,Market,Gift Shop,Bookstore,Vegetarian / Vegan Restaurant,Coffee Shop,American Restaurant
37,Peoria,Illinois,3.0,116513.0,40.693649,-89.588986,7,Coffee Shop,American Restaurant,Pizza Place,...,Bar,Chinese Restaurant,Supermarket,Cosmetics Shop,Sporting Goods Shop,Park,Theater,Smoothie Shop,Tex-Mex Restaurant,College Rec Center
38,Columbia,Missouri,34.0,115276.0,38.951705,-92.334072,7,Coffee Shop,Sandwich Place,Pizza Place,...,Bakery,Discount Store,Auto Dealership,ATM,Italian Restaurant,Steakhouse,Brewery,Burger Joint,Grocery Store,Sports Bar


In [259]:
# create map
cluster7 = folium.Map(location=[location.latitude, location.longitude], zoom_start=5)

# set color scheme for the clusters
x = np.arange(k_clusters)
ys = [i + x + (i*x)**2 for i in range(k_clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df7['Latitude'], df7['Longitude'], df7['City'], df7['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(cluster7)
       
cluster7

In [260]:
%%markdown

## Cluster 8


## Cluster 8


In [261]:
df8=df_merge.loc[df_merge['Cluster Labels'] == 8]
df8

,City,State,% Growth (2000 - 2013),Population,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,...,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
0,Chicago,Illinois,-6.1,2718782.0,41.878114,-87.629798,8,Hotel,Grocery Store,Park,...,Steakhouse,Sushi Restaurant,American Restaurant,Mediterranean Restaurant,Music Venue,Cocktail Bar,Garden,Sandwich Place,Mexican Restaurant,Gastropub
1,Indianapolis,Indiana,7.8,843393.0,39.768403,-86.158068,8,Bar,Hotel,Mexican Restaurant,...,New American Restaurant,Grocery Store,Plaza,Café,Steakhouse,Concert Hall,Breakfast Spot,Pizza Place,Park,Deli / Bodega
3,Detroit,Michigan,-27.1,688701.0,42.331427,-83.045754,8,Coffee Shop,Park,Hotel,...,Restaurant,Lounge,Diner,Bar,Burger Joint,Italian Restaurant,Outdoor Sculpture,Café,Waterfront,New American Restaurant
4,Milwaukee,Wisconsin,0.3,599164.0,43.038902,-87.906474,8,Hotel,Coffee Shop,Park,...,Burger Joint,Bar,BBQ Joint,Museum,New American Restaurant,Sandwich Place,Sushi Restaurant,Other Great Outdoors,Concert Hall,Wine Bar
7,Minneapolis,Minnesota,4.5,400070.0,44.977753,-93.265011,8,Hotel,Theater,Park,...,American Restaurant,Breakfast Spot,Plaza,Grocery Store,Salon / Barbershop,Scandinavian Restaurant,Event Space,Liquor Store,Vietnamese Restaurant,Wine Shop
8,Cleveland,Ohio,-18.1,390113.0,41.499320,-81.694361,8,Hotel,Coffee Shop,Steakhouse,...,Italian Restaurant,Deli / Bodega,BBQ Joint,Performing Arts Venue,Burger Joint,Museum,Wings Joint,Souvenir Shop,Roof Deck,Beer Bar
10,St. Louis,Missouri,-8.2,318416.0,38.627003,-90.199404,8,Hotel,Brewery,Pizza Place,...,Pub,Hockey Arena,Lounge,New American Restaurant,Donut Shop,Coffee Shop,Steakhouse,American Restaurant,French Restaurant,Breakfast Spot
11,Cincinnati,Ohio,-10.1,297517.0,39.103118,-84.512020,8,Bar,Coffee Shop,Park,...,Hotel,Art Museum,Performing Arts Venue,Bakery,Café,Brewery,Pizza Place,Steakhouse,Ice Cream Shop,Vietnamese Restaurant
12,St. Paul,Minnesota,2.8,294873.0,44.953703,-93.089958,8,Brewery,Pizza Place,Italian Restaurant,...,Bar,Coffee Shop,Southern / Soul Food Restaurant,Museum,Liquor Store,Dessert Shop,Athletics & Sports,Breakfast Spot,Jazz Club,Farmers Market
16,Madison,Wisconsin,15.8,243344.0,43.073052,-89.401230,8,Coffee Shop,Hotel,Bar,...,Plaza,Ice Cream Shop,Deli / Bodega,Sandwich Place,Sushi Restaurant,Steakhouse,Record Shop,French Restaurant,College Football Field,Garden


In [262]:
# create map
cluster8 = folium.Map(location=[location.latitude, location.longitude], zoom_start=5)

# set color scheme for the clusters
x = np.arange(k_clusters)
ys = [i + x + (i*x)**2 for i in range(k_clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df8['Latitude'], df8['Longitude'], df8['City'], df8['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(cluster8)
       
cluster8

In [263]:
%%markdown

## Cluster 9


## Cluster 9


In [264]:
df9=df_merge.loc[df_merge['Cluster Labels'] == 9]
df9

,City,State,% Growth (2000 - 2013),Population,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,...,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
13,Toledo,Ohio,-10.0,282313.0,41.663938,-83.555212,9,Discount Store,Fast Food Restaurant,Bar,...,Convenience Store,Grocery Store,Theater,Mexican Restaurant,Sandwich Place,Brewery,Art Museum,Park,Sushi Restaurant,Baseball Stadium
22,Springfield,Missouri,7.8,164122.0,37.208957,-93.292299,9,Bar,Coffee Shop,Pizza Place,...,Theater,Gas Station,Video Store,Brewery,Café,Grocery Store,Chinese Restaurant,Bakery,Sports Bar,Donut Shop
28,Warren,Michigan,-2.3,134873.0,42.514457,-83.014653,9,Discount Store,Sandwich Place,Coffee Shop,...,Gas Station,Fast Food Restaurant,Chinese Restaurant,Liquor Store,Cosmetics Shop,Italian Restaurant,Ice Cream Shop,Convenience Store,Mexican Restaurant,Donut Shop
30,Sterling Heights,Michigan,5.2,131224.0,42.580312,-83.030203,9,Sandwich Place,Fast Food Restaurant,Coffee Shop,...,Salon / Barbershop,Bank,Shipping Store,Discount Store,Mexican Restaurant,Disc Golf,Racetrack,Rental Car Location,Nightclub,Casino
36,Springfield,Illinois,4.2,117006.0,39.781721,-89.650148,9,Bar,Coffee Shop,Pizza Place,...,Theater,Gas Station,Video Store,Brewery,Café,Grocery Store,Chinese Restaurant,Bakery,Sports Bar,Donut Shop
47,Flint,Michigan,-20.0,99763.0,43.012527,-83.687456,9,Sandwich Place,Café,Chinese Restaurant,...,Theater,Pharmacy,Pub,Diner,College Bookstore,Food Truck,Gas Station,Taco Place,Fried Chicken Joint,Performing Arts Venue
83,Canton,Ohio,-10.3,72535.0,40.798947,-81.378447,9,Sandwich Place,Bar,Pharmacy,...,Art Museum,Fried Chicken Joint,Basketball Stadium,Greek Restaurant,Grocery Store,Brewery,Candy Store,History Museum,Restaurant,Football Stadium
87,Muncie,Indiana,-0.7,70316.0,40.193377,-85.386360,9,Fast Food Restaurant,Pizza Place,Sandwich Place,...,Café,Grocery Store,Theater,Museum,Ice Cream Shop,Bookstore,Deli / Bodega,Fried Chicken Joint,Student Center,Liquor Store
98,Youngstown,Ohio,-20.2,65184.0,41.099780,-80.649519,9,Bar,Fast Food Restaurant,Bank,...,Brewery,Storage Facility,Burger Joint,Science Museum,Rental Car Location,Concert Hall,Convenience Store,Pharmacy,Bakery,Park
107,Taylor,Michigan,-6.3,61817.0,42.240872,-83.269651,9,Sandwich Place,Fast Food Restaurant,Pizza Place,...,Video Store,Diner,Ice Cream Shop,Fried Chicken Joint,Convenience Store,Liquor Store,Supermarket,Gaming Cafe,Market,Supplement Shop


In [265]:
# create map
cluster9 = folium.Map(location=[location.latitude, location.longitude], zoom_start=5)

# set color scheme for the clusters
x = np.arange(k_clusters)
ys = [i + x + (i*x)**2 for i in range(k_clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df9['Latitude'], df9['Longitude'], df9['City'], df9['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(cluster9)
       
cluster9

In [266]:
%%markdown

## Cluster 10


## Cluster 10


In [267]:
df10=df_merge.loc[df_merge['Cluster Labels'] == 10]
df10

,City,State,% Growth (2000 - 2013),Population,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,...,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue


In [268]:
# create map
cluster10 = folium.Map(location=[location.latitude, location.longitude], zoom_start=5)

# set color scheme for the clusters
x = np.arange(k_clusters)
ys = [i + x + (i*x)**2 for i in range(k_clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df10['Latitude'], df10['Longitude'], df10['City'], df10['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(cluster10)
       
cluster10